# Profile with tides

In [ ]:
#Cell 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import detrend, periodogram, find_peaks, windows


In [ ]:
#Cell 2
details_path  = r"C:\Users\aalehrma\Documents\HARD_DRIVE_COPY\Dotson\Ribs\profiles\Drumlin1 rib profiles\1-47_rollingwindow7_ribID_details.xlsx"
sheet_name    = "Sheet1"
amp_col       = "height"
spacing_col   = "spacing"
sampling_interval = 1

details_df = pd.read_excel(details_path, sheet_name=sheet_name)


In [ ]:
#Cell 2a
import numpy as np
from scipy.signal import detrend

# --- Height detrending ---
details_df['height_linreg'] = detrend(details_df['height'].values, type='linear')

details_df['height_smooth100'] = (
    details_df['height_linreg']
      .rolling(window=100, center=True, min_periods=1)
      .mean()

)
details_df['height_detrended_filtered'] = (
    details_df['height_linreg']
    - details_df['height_smooth100']
)

# --- Spacing detrending ---
spacing_raw = details_df['spacing'].values.astype(float)
valid_idx = np.isfinite(spacing_raw)

spacing_clean = spacing_raw[valid_idx]
spacing_linreg = detrend(spacing_clean, type='linear')

spacing_linreg_full = np.full_like(spacing_raw, np.nan)
spacing_linreg_full[valid_idx] = spacing_linreg
details_df['spacing_linreg'] = spacing_linreg_full

details_df['spacing_smooth100'] = (
    details_df['spacing_linreg']
      .rolling(window=100, center=True, min_periods=1)
      .mean()
)

details_df['spacing_detrended_filtered'] = (
    details_df['spacing_linreg']
    - details_df['spacing_smooth100']
)


In [ ]:
# ---- Cell 3: Load slope/aspect/distance sheet ----
import pandas as pd
from scipy.spatial import cKDTree

slope_path  = r"C:\Users\aalehrma\Documents\HARD_DRIVE_COPY\Dotson\Ribs\profiles\Drumlin1 rib profiles\1-47_slopeaspect.xlsx"
slope_sheet = "Sheet1"

slope_df = pd.read_excel(slope_path, sheet_name=slope_sheet)
slope_df = slope_df.rename(columns={'X':'latitude', 'Y':'longitude'})

slope_pts  = np.column_stack((slope_df['latitude'], slope_df['longitude']))
tree       = cKDTree(slope_pts)
detail_pts = np.column_stack((details_df['latitude'], details_df['longitude']))
_, idx     = tree.query(detail_pts)

details_df['slope_value']  = slope_df['slope_value'].values[idx]
details_df['aspect_value'] = slope_df['aspect_value'].values[idx]
details_df['distance']     = slope_df['distance'].values[idx]

print(details_df[['latitude','longitude','distance','slope_value','aspect_value']].head())


In [ ]:
# ---- Cell 3a: nearest‐neighbour merge of slope/aspect & distance by lat/lon ----
from scipy.spatial import cKDTree
import numpy as np

# 1) rename slope‐sheet coords to match details_df
slope_df = slope_df.rename(columns={'X':'latitude', 'Y':'longitude'})

# 2) build KD‐tree on the slope locations
slope_pts = np.column_stack((slope_df['latitude'], slope_df['longitude']))
tree      = cKDTree(slope_pts)

# 3) query the nearest slope point for each rib location
detail_pts = np.column_stack((details_df['latitude'], details_df['longitude']))
_, idx     = tree.query(detail_pts)

# 4) pull in slope, aspect, AND distance
details_df['slope_value']  = slope_df['slope_value'].values[idx]
details_df['aspect_value'] = slope_df['aspect_value'].values[idx]
details_df['distance']     = slope_df['distance'].values[idx]   # <— bring in distance, too!

print(details_df[['latitude','longitude','distance','slope_value','aspect_value']].head())


In [ ]:
# --- Rolling slope window (for bias correction) ---
details_df['slope_roll5'] = (
    details_df['slope_value']
      .rolling(window=3, center=True, min_periods=1)
      .mean()
)

# --- Apply slope correction to height only ---
details_df['bias_rib'] = details_df['slope_roll5'] * details_df['spacing']
details_df['height_detrended_rib'] = (
    -(details_df['height'] - details_df['bias_rib'])
)

# --- For spacing, skip slope correction ---
# Just carry forward the detrended + filtered result from before
details_df['spacing_detrended_rib'] = details_df['spacing_detrended_filtered']


In [ ]:
# Cell 3.1 tides
import os
import pyproj
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

# import tide programs
import pyTMD.io
import pyTMD.tools
import datetime

In [ ]:
#cell 3.1a
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.value = 'CATS2008-v2023'
TMDwidgets.directory.value = "."

# Initialise the model
model = pyTMD.io.model(TMDwidgets.directory.value,
    compressed=TMDwidgets.compress.value
   ).elevation(TMDwidgets.model.value)

In [ ]:
# Cell 3.1b Set model parameters
lon = -112.4
lat = -74.4
time_range = np.arange(datetime.datetime(2020,3,22), datetime.datetime(2020,9,4), datetime.timedelta(hours=1))
model = pyTMD.io.model(verify=False).elevation(TMDwidgets.model.value)
model.parse_constituents()
# calculate tide elevations
tide = pyTMD.compute.tide_elevations(lon, lat, time_range,
    DIRECTORY=TMDwidgets.directory.value, TYPE='time series', 
    MODEL=TMDwidgets.model.value, GZIP=TMDwidgets.compress.value,
    EPSG=4326, TIME='datetime', EXTRAPOLATE=True, CUTOFF=20).squeeze()

In [ ]:
#Cell 3.1c Make dataframe of tides
df_tide = pd.DataFrame({"datetime": time_range, "tide (m)": tide})

# Calculate day number for convenience
df_tide['days'] = (df_tide['datetime'] - df_tide['datetime'][0]).dt.total_seconds() / (24 * 60 * 60)

# Write to file
df_tide.to_csv("tide_amp.csv", index=False)


In [ ]:
from datetime import timedelta
import matplotlib.transforms as mtrans
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import numpy as np

# Apply Gaussian convolution to rib height
height_clean = details_df['height_detrended_rib'].dropna()
smoothed_height = gaussian_filter1d(height_clean, sigma=2.5)

# Reinsert into original DataFrame with NaNs preserved
details_df['height_rib_gaussconv'] = np.nan
details_df.loc[height_clean.index, 'height_rib_gaussconv'] = smoothed_height

fig, ax1 = plt.subplots(figsize=(20, 12))
offset = 100

# Alternating stripes
start_full = df_tide['datetime'].min()
end_full = df_tide['datetime'].max()
delta = timedelta(days=7)
current, i = start_full, 0
while current < end_full:
    nxt = current + delta
    ax1.axvspan(current, nxt, ymin=0.0, ymax=0.02,
                facecolor=('lightgrey' if i % 2 == 0 else 'beige'),
                transform=ax1.get_xaxis_transform(), zorder=0)
    current, i = nxt, i + 1

# Monthly triangles
month_len, tri_dates = timedelta(days=30.4), []
t = start_full
while t < end_full:
    tri_dates.append(t)
    t += month_len
trans = mtrans.blended_transform_factory(ax1.transData, ax1.transAxes)
for td in tri_dates:
    ax1.scatter(td, 0.0, marker='^', s=50, color='black',
                transform=trans, clip_on=False, zorder=1)

# Times
times = df_tide['datetime'].iloc[offset: offset + len(details_df) * 24: 24]
times_shifted = times - pd.Timedelta(days=-2)

# Bar width
if len(times_shifted) > 1:
    bar_width = (times_shifted.iloc[1] - times_shifted.iloc[0]) * 0.8
else:
    bar_width = timedelta(days=1) * 0.8

# Scale for left plot (height + tides)
scale_rib = tide.max() / details_df['height_detrended_rib'].max()

# Prepare spacing (Gaussian convolution, negative direction)
spacing_first = details_df['spacing_detrended_filtered'].iloc[0]
spacing_shifted = -details_df['spacing_detrended_filtered'] - spacing_first

# Drop NaNs
spacing_clean = spacing_shifted.dropna()
spacing_smoothed = np.full_like(spacing_shifted, np.nan, dtype=float)
spacing_smoothed_std = np.full_like(spacing_shifted, np.nan, dtype=float)

# Gaussian smoothing (smoother than rolling)
spacing_smoothed[spacing_clean.index] = gaussian_filter1d(spacing_clean, sigma=1.75)
spacing_std = spacing_shifted.rolling(window=3, center=True, min_periods=1).std()
spacing_smoothed_std[spacing_clean.index] = gaussian_filter1d(spacing_std.dropna(), sigma=1.5)

# LEFT AXIS: tides + rib height
ax1.plot(df_tide['datetime'], tide, color='grey', label='tides')
ax1.bar(times_shifted,
        details_df['height_detrended_rib'] * scale_rib,
        width=bar_width,
        bottom=0,
        color='#e37800', alpha=0.3, label='rib height')
# ✅ Plot Gaussian convolution smoothed line
ax1.plot(times_shifted,
         details_df['height_rib_gaussconv'] * scale_rib,
         color='#e37800', lw=2, label='rib height (m')

ax1.set_ylabel('Amplitude (m)', color='#e37800', size=18)

# Get primary y-limits
y1_min, y1_max = ax1.get_ylim()
target_left_value = -0.75
zero_ratio = (target_left_value - y1_min) / (y1_max - y1_min)

# RIGHT AXIS: spacing (independent scale)
ax2 = ax1.twinx()
ax2.plot(times_shifted, spacing_smoothed, color='#001f5b', lw=2, label='spacing (mean)')
ax2.fill_between(times_shifted,
                 spacing_smoothed - spacing_smoothed_std,
                 spacing_smoothed + spacing_smoothed_std,
                 color='#b9e3f3', alpha=0.3, label='spacing ±1σ band')
ax2.set_ylabel('Spacing (m)', color='#001f5b', size=18)

spacing_mean = np.nanmean(spacing_smoothed)
ax2.axhline(spacing_mean, color='#c10000', lw=1.5, linestyle='--', label='mean spacing')


# Align ax2 zero with a value on ax1 (e.g., -0.75), and allow full vertical range of spacing
ax2_min, ax2_max = ax2.get_ylim()
ax2_span = ax2_max - ax2_min
new_ax2_min = -ax2_span * zero_ratio
new_ax2_max = ax2_span * (1 - zero_ratio)

# Expand top and bottom a bit to ensure full envelope is visible
padding = 0.1 * ax2_span
ax2.set_ylim(new_ax2_min - padding, new_ax2_max + padding)


# ✅ Sync x-axis limits
xlims = ax1.get_lines()[0].get_xdata()
xmin = xlims.min()
xmax = xlims.max()
ax1.set_xlim(xmin, xmax)
ax2.set_xlim(xmin, xmax)

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2,
           loc='center left',
           bbox_to_anchor=(1.12, 0.5),
           frameon=False,
           fontsize=18)

ax1.set_xlabel('Landform in series', size= 18)

# Tick every 15 landforms (i.e., days)
tick_spacing = 50
tick_locs = times_shifted[::tick_spacing]
tick_labels = np.arange(0, len(times_shifted) + 1, tick_spacing)

ax1.set_xticks(tick_locs)
ax1.set_xticklabels(tick_labels)

# Explicit date limits
ax1.set_xlim(pd.Timestamp('2020-03-28'), pd.Timestamp('2020-07-26'))
ax2.set_xlim(pd.Timestamp('2020-03-28'), pd.Timestamp('2020-07-26'))

ax1.tick_params(axis='y', labelcolor='#e37800', labelsize=14)
ax2.tick_params(axis='y', labelcolor='#001f5b', labelsize=14)
ax1.tick_params(axis='x', labelsize=14)

plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("rib_spacing_plot.png", dpi=300, bbox_inches='tight')
plt.show()
